<a href="https://colab.research.google.com/github/arshad62/learn-agentic-ai/blob/main/06b_crew_ai/05_augmented_llms/crewai_crews_advance_concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os

from google.colab import userdata

os.environ['GEMINI_API_KEY'] =  userdata.get('GEMINI_API_KEY')
#os.environ['OPENAI_API_KEY'] =  userdata.get('OPENAI_API_KEY')

In [3]:
!pip install -Uq crewai crewai-tools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.9/545.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 28.2 MB/s 

In [5]:
!pip install --upgrade chromadb
!pip install --upgrade tokenizers

  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.5.23
    Uninstalling chromadb-0.5.23:
      Successfully uninstalled chromadb-0.5.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.127 requires chromadb<0.6.0,>=0.5.10, but you have chromadb 0.6.3 which is incompatible.


In [6]:
import nest_asyncio
nest_asyncio.apply()

In [7]:
from crewai.flow.flow import Flow, start, listen


class MyFlow(Flow):

  @start()
  def function1(self):
    print("step1")

  @listen(function1)
  def function2(self):
    print("step2")

obj = MyFlow()
obj.kickoff()

 Flow started with ID: 6c3ff93a-d394-4960-a591-21df4e7c357d
step1
step2


In [8]:
from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-2.0-flash",
)

llm1.call("What is the capital of France?")

'The capital of France is **Paris**.\n'

In [19]:
google_embedder = {
    "provider": "google",
    "config": {
         "model": "models/text-embedding-004",
         "api_key": userdata.get('GEMINI_API_KEY'),
         }
}

In [11]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

# Create a knowledge source
content = "Users name is Arshad. He is 62 years old and lives in Doha, Qatar. He is working as Telecome Consultant at MoD"
string_source = StringKnowledgeSource(
    content=content,
)


# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    memory=True,
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[string_source], # Enable knowledge by adding the sources here. You can also add more sources to the sources list.
    embedder=google_embedder

)

result = crew.kickoff(inputs={"question": "What city does Arshad live in and how old is he?"})

# Agent: About User
## Task: Answer the following questions about the user: What city does Arshad live in and how old is he?


# Agent: About User
## Final Answer: 
Arshad lives in Doha, Qatar, and he is 62 years old.




In [12]:
import os
os.environ['CREWAI_STORAGE_DIR']='/my_crew1'

In [13]:
from crewai import LLM

llm1 = LLM(
    model="gemini/gemini-2.0-flash",
)

In [14]:
from crewai import Crew, Process
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
# from crewai.memory.storage import LTMSQLiteStorage, RAGStorage
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from crewai.memory.storage.rag_storage import RAGStorage

from typing import List, Optional

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    memory = True,
    # Long-term memory for persistent storage across sessions
    long_term_memory = LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="./my_crew2/long_term_memory_storage1.db"
        )
    ),
    # Short-term memory for current context using RAG
    short_term_memory = ShortTermMemory(
        storage = RAGStorage(
                embedder_config=google_embedder,
                type="short_term",
                path="./my_crew2/short_term1/"
            )
        ),

    # Entity memory for tracking key information about entities
    entity_memory = EntityMemory(
        storage=RAGStorage(
            embedder_config=google_embedder,
            type="short_term",
            path="./my_crew2/entity1/"
        )
    ),
    verbose=True,
)

# Integrating Mem0 for Enhanced User Memory


In [15]:
import os
from crewai import Crew, Process
from mem0 import MemoryClient

# Set environment variables for Mem0
os.environ["MEM0_API_KEY"] = userdata.get('MEM0_API_KEY')

# Step 1: Record preferences based on past conversation or user input
client = MemoryClient()
# messages = [
#     {"role": "user", "content": "Hi there! I'm planning a vacation and could use some advice."},
#     {"role": "assistant", "content": "Hello! I'd be happy to help with your vacation planning. What kind of destination do you prefer?"},
#     {"role": "user", "content": "I am more of a beach person than a mountain person."},
#     {"role": "assistant", "content": "That's interesting. Do you like hotels or Airbnb?"},
#     {"role": "user", "content": "I like Airbnb more."},
# ]
# client.add(messages, user_id="john")

# Step 2: Create a Crew with User Memory

# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,
)
task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    process = Process.sequential,
    verbose=True,
    memory=True,
    memory_config={
        "provider": "mem0",
        "config": {"user_id": "john"},
    },
)

crew.kickoff(inputs={"question": "What is your favorite vacation destination?"})

# Agent: About User
## Task: Answer the following questions about the user: What is your favorite vacation destination?


# Agent: About User
## Final Answer: 
As an AI, I do not have personal preferences or the ability to travel, so I do not have a favorite vacation destination.




/usr/local/lib/python3.11/dist-packages/mem0/client/main.py:32: DeprecationWarning: Using default output format 'v1.0' is deprecated and will be removed in version 0.1.70. Please use output_format='v1.1' for enhanced memory details. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/mem0/client/main.py:32: DeprecationWarning: Using default output format 'v1.0' is deprecated and will be removed in version 0.1.70. Please use output_format='v1.1' for enhanced memory details. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


CrewOutput(raw='As an AI, I do not have personal preferences or the ability to travel, so I do not have a favorite vacation destination.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Answer the following questions about the user: What is your favorite vacation destination?', name=None, expected_output='An answer to the question.', summary='Answer the following questions about the user: What is your...', raw='As an AI, I do not have personal preferences or the ability to travel, so I do not have a favorite vacation destination.', pydantic=None, json_dict=None, agent='About User', output_format=<OutputFormat.RAW: 'raw'>)], token_usage=UsageMetrics(total_tokens=251, prompt_tokens=177, cached_prompt_tokens=0, completion_tokens=74, successful_requests=1))

# Tools

In [1]:
!pip install --upgrade transformers
!pip install --upgrade tokenizers
!pip install -Uq 'crewai[tools]'

  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.127 requires chromadb<0.6.0,>=0.5.10, but you have chromadb 0.6.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.49.0 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.


In [19]:
!pip install --upgrade chromadb

  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
Using cached chromadb-0.6.3-py3-none-any.whl (611 kB)
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.5.23
    Uninstalling chromadb-0.5.23:
      Successfully uninstalled chromadb-0.5.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.127 requires chromadb<0.6.0,>=0.5.10, but you have chromadb 0.6.3 which is incompatible.


In [5]:
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [6]:
embedder = {
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": userdata.get('GEMINI_API_KEY'),
        }
    }

In [12]:
!pip install -Uq langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [14]:
from crewai import LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm1=LLM(model="gemini/gemini-1.5-flash")

In [20]:
!pip install openai
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

# Set up API keys
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY') # serper.dev API key
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
# Example with WebsiteSearchTool
web_rag_tool = WebsiteSearchTool(embedder_config=google_embedder)

# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],
    verbose=True,
    llm=llm1
)

writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool, file_tool],
    verbose=True,
    llm=llm1
)

# Define tasks
research = Task(
    description='Research the latest trends in the AI industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments in the AI industry with a unique perspective on their significance.',
    agent=researcher
)

write = Task(
    description='Write an engaging blog post about the AI industry, based on the research analyst’s summary. Draw inspiration from the latest blog posts in the directory.',
    expected_output='A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.',
    agent=writer,
    output_file='blog-posts/new_post.md'  # The final blog post will be saved here
)

# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=True,  # Enable planning feature
    planning_llm=llm1
)

# Execute tasks
crew.kickoff()

KeyError: 'OPENAI_API_KEY'

In [ ]:
# prompt: import markdown for jupyter display

from IPython.display import Markdown

def display_markdown_file(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    display(Markdown(content))

# Example usage: Assuming 'blog-posts/new_post.md' is the file generated by your code
display_markdown_file('blog-posts/new_post.md')


# The AI Revolution: Top Trends Shaping the Future

The artificial intelligence industry is experiencing unprecedented growth and innovation, transforming businesses and daily life as we know it. From self-driving cars to virtual assistants, AI's influence is undeniable, and the pace of development shows no signs of slowing down. Staying informed about the latest trends is crucial for anyone looking to understand and leverage the power of AI.

Currently, three major trends are taking center stage. First, advancements in generative AI are democratizing content creation and problem-solving. Second, the growing emphasis on ethical AI and responsible development is ensuring that AI systems are fair, transparent, and beneficial to society. And finally, AI is revolutionizing healthcare, offering new possibilities for diagnostics, treatment, and patient care.

What makes these trends particularly significant is their potential to create a more accessible, trustworthy, and equitable future. Generative AI is empowering non-technical users to harness the power of AI, while ethical AI is building trust and ensuring the long-term sustainability of AI adoption. Moreover, AI's transformative impact on healthcare has the potential to democratize access to quality medical services and improve patient outcomes globally, especially with the convergence of AI and robotics.

Looking ahead, the AI industry promises even more groundbreaking developments. As AI technologies continue to mature, we can expect to see even greater integration into our daily lives, driving innovation and creating new opportunities across various sectors. By embracing these advancements and addressing the ethical considerations that come with them, we can unlock the full potential of AI and create a future where technology benefits all of humanity.
```

## Creating your own Tools

In [ ]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

class MyToolInput(BaseModel):
    """Input schema for MyCustomTool."""
    student_name: str = Field(..., description="Student name")
    student_roll_no : int = Field(..., description="student id")

class PiaicStudentCard(BaseTool):
    name: str = "Piaic student card generator"
    description: str = "this function will create Piaic student card"
    args_schema: Type[BaseModel] = MyToolInput

    def _run(self, student_name: str, student_roll_no: int ) -> str:
        # Your tool's logic here
        return f"""PIAIC student card
student name: {student_name}
student roll no: {student_roll_no}
Pakistan zindabd!
        """

In [ ]:
from crewai.tools import tool

@tool("PIAIC fee update")
def my_tool(roll_no: int) -> dict | str:
    """this function search piaic student fee updates, it will required roll no of PIAIC student"""
    #database

    data = {100:'paid',
         200:'unpaid'}


    status = data.get(roll_no)

    if status:
      return {"status": status}
    else:
      return "student not found"





In [ ]:
from crewai import Agent, Task, Crew

card = PiaicStudentCard()


piaic_manager = Agent(
    role="PIAIC manager",
    goal = "Manage all quries regarding PIAIC and you will use only relevant tools for student query",
    backstory="""You are a master at understanding people and their preferences.""",
    tools=[card, my_tool],
    verbose=True,
    llm=llm1
)

piaic_card_creator = Task(
    description="you will be responsible for all PIAIC relevant operations, student query '{query}' you must be know how to answer his question based on final context",
    expected_output="final query answer only",
    agent=piaic_manager
)

crew = Crew(
    agents=[piaic_manager],
    tasks=[piaic_card_creator],
    verbose=True
)

result = crew.kickoff(inputs={
    "query":"I'm PIAIC student my name is Muhammad Qasim and my roll number is 100, can you create my student card."
})

print(result)

# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student my name is Muhammad Qasim and my roll number is 100, can you create my student card.' you must be know how to answer his question based on final context


# Agent: PIAIC manager
## Thought: The user is asking to generate a student card. I should use the "Piaic student card generator" tool to generate the card. I need to extract the student's name and roll number from the query.
## Using tool: Piaic student card generator
## Tool Input: 
"{\"student_name\": \"Muhammad Qasim\", \"student_roll_no\": 100}"
## Tool Output: 
PIAIC student card
student name: Muhammad Qasim
student roll no: 100
Pakistan zindabd!
        


# Agent: PIAIC manager
## Final Answer: 
PIAIC student card
student name: Muhammad Qasim
student roll no: 100
Pakistan zindabd!


PIAIC student card
student name: Muhammad Qasim
student roll no: 100
Pakistan zindabd!


In [ ]:
result = crew.kickoff(inputs={
    "query":"I'm PIAIC student  my roll number is 100, can you share my fee updates?."
})

print(result)

# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student  my roll number is 100, can you share my fee updates?.' you must be know how to answer his question based on final context


# Agent: PIAIC manager
## Using tool: PIAIC fee update
## Tool Input: 
"{\"roll_no\": 100}"
## Tool Output: 
{'status': 'paid'}


# Agent: PIAIC manager
## Final Answer: 
your fee is paid


your fee is paid


In [ ]:
result = crew.kickoff(inputs={
    "query":"I'm PIAIC student  my roll number is 200, can you share my fee updates?."
})

print(result)

# Agent: PIAIC manager
## Task: you will be responsible for all PIAIC relevant operations, student query 'I'm PIAIC student  my roll number is 200, can you share my fee updates?.' you must be know how to answer his question based on final context


# Agent: PIAIC manager
## Using tool: PIAIC fee update
## Tool Input: 
"{\"roll_no\": 200}"
## Tool Output: 
{'status': 'unpaid'}


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Piaic student card generator
Tool Arguments: {'student_name': {'description': 'Student name', 'type': 'str'}, 'student_roll_no': {'description': 'student id', 'type': 'int'}}
Tool Description: this function will create Piaic student card
Tool Name: PIAIC fee update
Tool Arguments: {'roll_no': {'description': None, 'type': 'int'}}
Tool Description: this function search piaic student fee updates, it will required roll no of PIAIC student

IMPORTANT: Use the following format in your response:

```
Thoug

# Process

In [ ]:
import os
from crewai import Agent, Task, Crew, Process

# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)



# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts2')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

# Define your agents
researcher = Agent(
    role="Researcher",
    goal="Conduct thorough research and analysis on AI and AI agents",
    backstory="You're an expert researcher, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently researching for a new client.",
    allow_delegation=False,
    tools=[search_tool,web_rag_tool],
    llm=llm1
)

writer = Agent(
    role="Senior Writer",
    goal="Create compelling content about AI and AI agents",
    backstory="You're a senior writer, specialized in technology, software engineering, AI, and startups. You work as a freelancer and are currently writing content for a new client.",
    allow_delegation=False,
    tools=[docs_tool,file_tool],
    llm=llm1
)

# Define your task
task = Task(
    description="Generate a list of 5 interesting ideas for an article, then write one captivating paragraph for each idea that showcases the potential of a full article on this topic. Return the list of ideas with their paragraphs and your notes.",
    expected_output="5 bullet points, each with a paragraph and accompanying notes.",
)

# Define the manager agent
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    allow_delegation=True,
    llm=llm1
)

# Instantiate your crew with a custom manager
crew = Crew(
    agents=[researcher, writer],
    tasks=[task],
    manager_agent=manager,
    function_calling_llm=llm1,
    process=Process.hierarchical,
    verbose=True

)

# Start the crew's work
result = crew.kickoff()
from IPython.display import Markdown

Markdown(result.raw)

# Agent: Project Manager
## Task: Generate a list of 5 interesting ideas for an article, then write one captivating paragraph for each idea that showcases the potential of a full article on this topic. Return the list of ideas with their paragraphs and your notes.


# Agent: Project Manager
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": {\"description\": \"Research potential topics for articles that would be interesting and engaging for a general audience. I need a diverse range of ideas, so explore different fields and current trends.\", \"type\": \"str\"}, \"context\": {\"description\": \"The goal is to generate 5 distinct and captivating article ideas that can be developed into full-length pieces. The target audience is broad, so topics should be accessible and engaging to a wide range of readers.\", \"type\": \"str\"}, \"coworker\": {\"description\": \"Researcher\", \"type\": \"str\"}}"
## Tool Output: 

I encountered an error while trying to use the tool. Thi

*   **Article Idea:** "AI Assistants: Friend or Foe?"

    **Captivating Paragraph:** Imagine a world where your every command is anticipated, your schedule flawlessly managed, and information instantly at your fingertips. This is the promise of AI assistants like Alexa, Siri, and Google Assistant, now woven into the fabric of our daily lives. But behind the veneer of convenience lurks a web of potential privacy breaches, data security vulnerabilities, and the creeping erosion of genuine human connection. Are we trading our autonomy for ease, and what is the true cost of this digital bargain?

    **Notes:** This article can explore the ethical implications of AI, interview experts on data privacy, and feature personal anecdotes of people's experiences with AI assistants.

*   **Article Idea:** "Beyond the Headlines: Understanding Climate Anxiety and Finding Hope"

    **Captivating Paragraph:** The weight of a warming planet bears down on us, a constant stream of dire headlines fueling a growing epidemic of climate anxiety. But beyond the fear and despair lies the power to act, to reclaim our hope and forge a more sustainable future. This article dives deep into the psychological impact of climate change, offering practical tools to cope with eco-related stress and empowering readers to become agents of positive change, one small step at a time.

    **Notes:** This article could include interviews with climate psychologists, showcase successful grassroots environmental initiatives, and offer actionable tips for reducing your carbon footprint.

*   **Article Idea:** "The Gig Economy Game: How to Thrive in the Age of Freelancing"

    **Captivating Paragraph:** The traditional 9-to-5 is fading, replaced by a dynamic landscape of freelancers, contractors, and independent consultants. The gig economy offers unprecedented freedom and flexibility, but also presents a unique set of challenges. From securing consistent work to managing finances and maintaining work-life balance, this article provides a roadmap for navigating the new world of work and building a thriving freelance career.

    **Notes:** This article could feature interviews with successful freelancers, offer practical tips for managing finances and taxes, and explore the legal and ethical considerations of the gig economy.

*   **Article Idea:** "The Power of Lifelong Learning: Staying Relevant in a Rapidly Changing World"

    **Captivating Paragraph:** In an era of relentless technological advancement, the ability to learn and adapt is no longer a luxury, but a necessity. The skills that are valued today may be obsolete tomorrow, making lifelong learning the key to staying relevant and thriving in a rapidly changing world. This article explores the transformative power of continuous learning, showcasing accessible resources and inspiring readers to embrace a growth mindset and unlock their full potential.

    **Notes:** This article could feature interviews with individuals who have successfully reinvented their careers through learning, highlight innovative online learning platforms, and offer practical tips for incorporating learning into your daily routine.

*   **Article Idea:** "Beyond Representation: What True Inclusion Really Means"

    **Captivating Paragraph:** We celebrate diversity, but are we truly inclusive? True inclusion goes beyond mere representation, demanding a deeper examination of unconscious biases, systemic inequalities, and the subtle microaggressions that can marginalize individuals and undermine a sense of belonging. This article delves into the complexities of creating truly inclusive environments in workplaces, schools, and communities, offering actionable strategies for fostering equity, empathy, and respect for all.

    **Notes:** This article could feature interviews with diversity and inclusion experts, showcase successful inclusion initiatives in different organizations, and offer practical tips for addressing unconscious bias and promoting cultural sensitivity.

# Advance Tools

In [ ]:
from pydantic import BaseModel, Field

class TeacherAssignmentOutPut(BaseModel):
  score :int  = Field(..., description="student assignment score")
  feedback : str = Field(..., description="teacher feedback")
  question : str = Field(..., description="question")
  answer : str = Field(..., description="python code for markdown format")

In [ ]:
from crewai import Agent
from crewai_tools import CodeInterpreterTool
from crewai import Task, Crew, Process

teacher = Agent(
    role="AI teacher",
    goal="you are Agentic AI teacher you have to check python assignment code submission.",
    backstory="you have to check student python code submission.",
    tools=[CodeInterpreterTool()],
    llm=llm1
)

assignment_check = Task(
    description="you are Agentic AI teacher you have to check python assignment code submission. question: '''{qestion}''' student solution : '''{solution}''' ",
    expected_output="final code is runing, assign number between 1-10",
    output_pydantic=TeacherAssignmentOutPut,
    agent=teacher # Added the missing agent here
)


crew = Crew(
    agents=[teacher],
    tasks=[assignment_check],
    process=Process.sequential,
    verbose=True,
)

question = "create function add two numbers"

solution = """
def add_two_number(num1 : int , num2 : int):
  return num1 + num2

"""

result = crew.kickoff(inputs={
    "qestion": question,
    "solution" : solution
})

print(result)

# Agent: AI teacher
## Task: you are Agentic AI teacher you have to check python assignment code submission. question: '''create function add two numbers''' student solution : '''
def add_two_number(num1 : int , num2 : int):
  return num1 + num2

''' 


# Agent: AI teacher
## Using tool: Code Interpreter
## Tool Input: 
"{\"code\": \"def add_two_number(num1 : int , num2 : int):\\n  return num1 + num2\\n\\nprint(add_two_number(5, 3))\", \"libraries_used\": []}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory')).
 Tool Code Interpreter accepts these inputs: Tool Name: Code Interpreter
Tool Arguments: {'code': {'description': 'Python3 code used to be interpreted in the Docker container. ALWAYS PRINT the final result and the output of the code', 'type': 'str'}, 'libraries_used': {'description': 'List of libraries used in the code with p

In [ ]:
result.to_dict()

{'score': 9,
 'feedback': 'The code is correct and defines a function that adds two numbers. The type hints are a good addition. A small improvement could be adding a docstring to explain what the function does.',
 'question': 'create function add two numbers',
 'answer': 'def add_two_number(num1 : int , num2 : int):\n  return num1 + num2'}

In [ ]:
for k,v in result.to_dict().items():
  print(str("*"*10) + str(k) + str("*"*10))
  display(v)

**********score**********


9

**********feedback**********


"The function correctly adds two numbers.  Consider adding docstrings for better readability and to explain the function's purpose."

**********question**********


'create function add two numbers'

**********answer**********


'def add_two_number(num1, num2):\n  return num1 + num2\n'